In [1]:
import matplotlib.pyplot as plt

# Branch-and-Bound algorithm for 2-dimensional plane

In [7]:
class Disk:
    
    def __init__(self, center):
        self.center = center
        
    def contains(self, point):
        # True if the distance between the center and the given point <= 1
        return ((self.center[0] - point[0]) ** 2 + (self.center[1] - point[1]) ** 2) ** 0.5 <= 1
    

d = Disk((0, 0))
print(d.contains((0.5, 0.5)))
print(d.contains((0.5, 4.5)))
print(d.contains((1, 0)))
print(d.contains((0, 1)))
print(d.contains((0, 1+0.000001)))

True
False
True
True
False


In [16]:
class BBTree:
    
    def __init__(self, P, Q):
        self.P = P
        self.Q = Q
        self.h = self.heuristic()
        print(self.h, P, Q)
        
    def set_parent(self, parent):
        self.parent = parent
        
    def set_children(self, children):
        self.children = children
    
    def heuristic(self):
        return len(self.Q)  # Remaining points to cover
    
    def split(self):
        # Split in order of P
        # Check leaf condition
        if len(Q) == 0:
            print("Leaf")
            return None
        elif len(P) == 0:
            print("No disks left.")
            return None
        
        else:
            current_disk = self.Q[0]
            remaining_points = set([point for point in self.P if not current_disk.contains(point)])
            left_child = BBTree(remaining_points, self.Q[1:])  # Disk is added to the solution
            right_child = BBTree(self.P[:], self.Q[1:])  # Disk not added, create a copy of the list of points
            return left_child, right_child
            
            
            
def branch_and_bound(P, Q):
    root = BBTree(P, Q)
    to_visit = [root]
    
    while len(to_visit) != 0:
        pass
    


P = [(1, 1), (2, 2), (3, 3)]
Q = [Disk((1, 1.2)), Disk((2.3, 2.1))]

tree = BBTree(P, Q)
tree.split()

2 [(1, 1), (2, 2), (3, 3)] [<__main__.Disk object at 0x000001FBC48BF700>, <__main__.Disk object at 0x000001FBC48BF550>]
1 {(3, 3), (2, 2)} [<__main__.Disk object at 0x000001FBC48BF550>]
1 [(1, 1), (2, 2), (3, 3)] [<__main__.Disk object at 0x000001FBC48BF550>]


(<__main__.BBTree at 0x1fbc498b430>, <__main__.BBTree at 0x1fbc48bf970>)